**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables =  [table.table_id for table in tables]# Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [ ]:
#q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [4]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,58545647,None,"<p>You can implement the <a href=""https://docs...",None,None,0,None,2019-10-24 16:35:51.947000+00:00,None,2019-10-24 16:35:51.947000+00:00,None,None,None,None,2541560,58545487,2,0,None,None
1,58545649,None,"<p>You may be having an issue with the ""stage""...",None,None,0,None,2019-10-24 16:35:59.377000+00:00,None,2019-10-24 16:35:59.377000+00:00,None,None,None,None,4434749,56565949,2,0,None,None
2,58545664,None,<p>I am not sure why you need that exactly but...,None,None,0,None,2019-10-24 16:36:39.870000+00:00,None,2019-10-24 16:36:39.870000+00:00,None,None,None,None,8343843,58545068,2,0,None,None
3,58545675,None,<pre><code>Object delegateObj = readField(valu...,None,None,1,None,2019-10-24 16:37:20.207000+00:00,None,2019-10-24 16:37:20.207000+00:00,None,None,None,None,12269981,57195785,2,0,None,None
4,58545677,None,<p>I had to remove the line</p>\n\n<pre><code>...,None,None,0,None,2019-10-24 16:37:51.253000+00:00,None,2019-10-24 16:37:51.253000+00:00,None,None,None,None,1775258,58428566,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [5]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,59030907,NLP Transformers: Best way to get a fixed sent...,<p>I'm loading a language model from torch hub...,None,2,0,None,2019-11-25 11:36:12.360000+00:00,2.0,2019-11-26 16:49:27.200000+00:00,2019-11-26 16:49:27.200000+00:00,None,3192021.0,None,3192021,None,1,4,machine-learning|deep-learning|nlp|pytorch|wor...,256
1,59123008,How to detect keypress in C++ on MacOS?,<p>I'm writing an assignment that needs to sho...,None,0,0,None,2019-12-01 06:13:36.580000+00:00,NaN,2019-12-01 06:13:36.580000+00:00,NaT,None,NaN,None,10166333,None,1,0,c++|events|sdl|keypress|keyboard-events,1
2,58858824,stabilizing Android command line app benchmark...,<p>I'm writing C++ module which later will be ...,None,0,0,None,2019-11-14 14:14:51.043000+00:00,NaN,2019-11-14 14:14:51.043000+00:00,NaT,None,NaN,None,5827390,None,1,0,android|performance-testing|benchmarking,2
3,58947708,How to find pixel/dp distance between 2 points...,<p>I need to draw limit lines on both x and y-...,None,0,0,None,2019-11-20 06:01:55.060000+00:00,NaN,2019-11-20 11:39:16.020000+00:00,2019-11-20 11:39:16.020000+00:00,None,1631967.0,None,7637587,None,1,-1,mpandroidchart,2
4,58968440,will mobile browser download sourcemap file?,"<p>I've read some article or question said ""so...",None,0,0,None,2019-11-21 06:19:37.400000+00:00,NaN,2019-11-21 06:19:37.400000+00:00,NaT,None,NaN,None,3983650,None,1,0,source-maps,2


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [6]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [7]:
# Your code here
questions_query = """
                  SELECT  id,
                          title,
                          owner_user_id,
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  54007400  IAM Custom Role for Inserting to Specific BigQ...      7393707.0
1  54299820  Assign incremental value to multiple entries w...      9941132.0
2  54109779           Firebase performance data with big query      7893736.0
3  54195300         Firebase: Filtrating Events key and values     10915857.0
4  54155753  Error using Dataproc and Spark BigQuery Connec...      2844527.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [20]:
# Your code here
answers_query = """
                SELECT  a.id,
                        a.body,
                        a.owner_user_id,
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                        ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """
    # In words: Select id, body and owner_user_id **FROM ANSWERES**, from the intersect
    # between the two datasets (which intersect by the variabels specified), with 
    # a condition (that bigquery thing)
               
# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

BadRequest: 400 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/52884766-1ce7-4ce9-882e-60d5ba6e3fd9?maxResults=0&location=US: Query exceeded limit for bytes billed: 10000000000. 23881318400 or higher required.

For a hint or the solution, uncomment the appropriate line below.

In [21]:
#q_4.hint()
#q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [ ]:
# Your code here
bigquery_experts_query = """
                    SELECT  a.owner_user_id as user_id,
                            COUNT(1) AS number_of_answers
                    FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                         ON q.id = a.parent_id
                    WHERE q.tags LIKE '%bigquery%'
                    GROUP BY a.owner_user_id
                    ORDER BY number_of_answers DESC
                         """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query,
                                         job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_5.hint()
#q_5.solution()

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback

Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*